In [2]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
#from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.model_selection import train_test_split
# Define data root directory
#data_dir = "./data/"


In [3]:
# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation

    
# Store csv file in a Pandas DataFrame
df = pd.read_csv('combined_labled_k6.csv')
df.head()

,samples,joint_effort[0],joint_effort[1],joint_effort[2],joint_effort[3],joint_effort[4],joint_effort[5],cluster
0,0,1.050352,0.036084,0.153670,-0.208722,-0.226335,0.071900,0
1,1,1.040706,0.052485,0.160204,-0.204533,-0.222889,0.070273,0
2,2,1.070528,0.033901,0.198342,-0.208156,-0.228432,0.070408,0
3,3,1.078726,0.032952,0.203791,-0.208763,-0.228608,0.071877,0
4,4,1.046257,0.048101,0.215115,-0.205263,-0.223011,0.071184,0


In [55]:
df.cluster

0        0
1        0
2        0
3        0
4        0
        ..
35833    0
35834    0
35835    0
35836    0
35837    0
Name: cluster, Length: 35838, dtype: int64

In [79]:
sc = MinMaxScaler()
label_sc = MinMaxScaler()
#data = sc.fit_transform(df.values)
# Obtaining the Scale for the labels(usage data) so that output can be re-scaled to actual value during evaluation
label_sc.fit(df.cluster.values.reshape(-1,1))
label_scalers = label_sc
print(label_scalers)
test_x = np.array(df.iloc[1000:1500,1:7])
test_y = np.array(df.iloc[1000:1500,7:8])


MinMaxScaler()


In [80]:
#from locale import currency
#from pickletools import float8


class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.window=5
    def __len__(self):
       
        Q = len(self.df)
        self.length=Q-self.window+1
        return self.length

   
    def __getitem__(self, idx):
        x=np.array(self.df)
        if idx>= 35830:
            idx=idx-self.window
            X = x[idx:idx+self.window,1:7]
            Y=  x[idx,7:8]
        else:
            X = x[idx:idx+self.window,1:7]
            Y=  x[idx+self.window,7:8]
        
        return X,Y


In [81]:
cs= CustomDataset(df)

train_l=round(int(cs.__len__())*0.8)
#print(train_l)
batch_size=25
val_l=int(cs.__len__())-train_l
train_set, val_set = torch.utils.data.random_split(cs, [train_l, val_l])
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, drop_last=True)
for each_x, each_y in train_loader:
    print(each_x.shape)
    break
    
    
val_loader = DataLoader(val_set, shuffle=True, batch_size=batch_size, drop_last=True)
for each_x, each_y in val_loader:
    each_x = torch.tensor(each_x,dtype=float)
    each_y = torch.tensor(each_y,dtype=float)
    
    break


torch.Size([25, 5, 6])


/tmp/ipykernel_10163/2797386379.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  each_x = torch.tensor(each_x,dtype=float)
/tmp/ipykernel_10163/2797386379.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  each_y = torch.tensor(each_y,dtype=float)


In [82]:
len(df)

35838

In [83]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [84]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        #print(x,h)
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [95]:
from cProfile import label


def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.time()
    for i in test_y:
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.time()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [86]:





len(val_set.dataset)

35834

In [87]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")
Lstm_model = train(train_loader, lr, model_type="LSTM")

Starting Training of GRU model
Epoch 1......Step: 200/1146....... Average Loss for Epoch: 0.2541107908450067
Epoch 1......Step: 400/1146....... Average Loss for Epoch: 0.2389455007435754
Epoch 1......Step: 600/1146....... Average Loss for Epoch: 0.22386216358902553
Epoch 1......Step: 800/1146....... Average Loss for Epoch: 0.20559164744598094
Epoch 1......Step: 1000/1146....... Average Loss for Epoch: 0.193593687061104
Epoch 1/5 Done, Total Loss: 0.18250919365879542
Total Time Elapsed: 35.41452932357788 seconds
Epoch 2......Step: 200/1146....... Average Loss for Epoch: 0.10780633241403847
Epoch 2......Step: 400/1146....... Average Loss for Epoch: 0.10042820368689717
Epoch 2......Step: 600/1146....... Average Loss for Epoch: 0.09824502304332175
Epoch 2......Step: 800/1146....... Average Loss for Epoch: 0.09560663290700176
Epoch 2......Step: 1000/1146....... Average Loss for Epoch: 0.0956929579161806
Epoch 2/5 Done, Total Loss: 0.0942174378329649
Total Time Elapsed: 26.89289355278015 sec

In [96]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor